# 第6章: 英語テキストの処理

英語のテキスト（nlp.txt）に対して，以下の処理を実行せよ．

## 50. 文区切り
(. or ; or : or ? or !) → 空白文字 → 英大文字というパターンを文の区切りと見なし，入力された文書を1行1文の形式で出力せよ．

In [10]:
!head -n 8 nlp.txt

Natural language processing
From Wikipedia, the free encyclopedia

Natural language processing (NLP) is a field of computer science, artificial intelligence, and linguistics concerned with the interactions between computers and human (natural) languages. As such, NLP is related to the area of humani-computer interaction. Many challenges in NLP involve natural language understanding, that is, enabling computers to derive meaning from human or natural language input, and others involve natural language generation.

History

The history of NLP generally starts in the 1950s, although work can be found from earlier periods. In 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence.


In [19]:
words=[] #一文ずつリストに格納される
with open("nlp.txt","r") as nlp_line:
    nlp_line.replace(".","<EOS>")

AttributeError: '_io.TextIOWrapper' object has no attribute 'replace'

In [18]:
words[0:8]

['Natural language processing',
 'From Wikipedia, the free encyclopedia',
 '',
 'Natural language processing (NLP) is a field of computer science, artificial intelligence, and linguistics concerned with the interactions between computers and human (natural) languages. As such, NLP is related to the area of humani-computer interaction. Many challenges in NLP involve natural language understanding, that is, enabling computers to derive meaning from human or natural language input, and others involve natural language generation.',
 '',
 'History',
 '',
 'The history of NLP generally starts in the 1950s, although work can be found from earlier periods. In 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence.']

In [75]:
#正規表現テスト
import re
test_doc="Hello,World.:;  How are you ? : "
'''
adict = {"." : " ",
                 ":":" ",
                 ";":" "}
'''
adict = {"(\.|,|;|:)" : "<EOS>",}
#multiple_replace(test_doc,adict)
res=multiple_replace_re(test_doc,adict)
res

'Hello<EOS>World<EOS><EOS><EOS>  How are you ? <EOS> '

In [81]:
test_doc.replace(".","<EOS>")

'Hello,World<EOS>:;  How are you ? : '

In [27]:
def multiple_replace(text,adict):  
    """ 
    一度に複数の文字列を置換する. 
    - text中からディクショナリのキーに合致する文字列を探し、対応の値で置換して返す 
    """  
    # マッチさせたいキー群を正規表現の形にする e.g) (a1|a2|a3...)  
    rx = re.compile('|'.join(map(re.escape,adict)))  
    def one_xlat(match):  
        return adict[match.group(0)]  
    return rx.sub(one_xlat, text)  

In [38]:
def multiple_replace_re(text,adict):
    rx = re.compile('|'.join(adict))
    
    def dedictkey(text):
        for key in adict.keys():
            if re.search(key,text):
                return key
    
    def one_xlat(match):
        return adict[dedictkey(match.group(0))]
    
    return rx.sub(one_xlat,text)

In [30]:
re.compile("|".join(adict))

re.compile(r'.|:|;', re.UNICODE)

In [31]:
adict.keys()

dict_keys(['.', ':', ';'])

In [82]:
test ="Hello,World. I have a pen."

In [83]:
adict = {"(\.|,|;|:)" : "<EOS>",}
result=multiple_replace_re(test,adict)


In [84]:
result

'Hello<EOS>World<EOS> I have a pen<EOS>'

In [88]:
adict = {"(\.|,|;|:)" : "<EOS>",}
with open("nlp.txt","r") as nlp_line:
    print(nlp_line.decode("latin-1"))
    result = multiple_replace_re(nlp_line.decode("latin-1"),adict)

AttributeError: '_io.TextIOWrapper' object has no attribute 'decode'

In [1]:
import re

def sentence_punc():
    ans = ""
    pattern = re.compile("(?P<last>[\.;:\?!]) (?P<first>[A-Z])")
    
    for line in open("nlp.txt", "r"):
        ans += re.sub(pattern, "\g<last>\n\g<first>", line)

    return ans


print(sentence_punc())

Natural language processing
From Wikipedia, the free encyclopedia

Natural language processing (NLP) is a field of computer science, artificial intelligence, and linguistics concerned with the interactions between computers and human (natural) languages.
As such, NLP is related to the area of humani-computer interaction.
Many challenges in NLP involve natural language understanding, that is, enabling computers to derive meaning from human or natural language input, and others involve natural language generation.

History

The history of NLP generally starts in the 1950s, although work can be found from earlier periods.
In 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence.

The Georgetown experiment in 1954 involved fully automatic translation of more than sixty Russian sentences into English.
The authors claimed that within three or five years, machine translation would b

In [3]:
test = "Hello,World This document is test. \n  Hello,World"
pattern = re.compile("(?P<last>[\.;:\?!]) (?P<first>[A-Z])")
re.sub(pattern,"\g<last>\n\g<first>",test)

'Hello,World This document is test. \n  Hello,World'

In [4]:
pattern.group("last")

AttributeError: '_sre.SRE_Pattern' object has no attribute 'group'